# Paris Metropolitan, an evolution

## Map generator
This notebook generates two of the three maps displayed on the website :
1. The map with the three different layers from the data retrieved from the old maps
2. The map displaying the growing network over time based on external data added to the database

In [2]:
#imports
import pandas as pd
import numpy as np
import os
import json
import folium
from folium import plugins
from scipy.ndimage import imread
import random
import copy
import collections
from collections import Counter
from collections import defaultdict
from itertools import filterfalse
import time
import ijson
import csv
import ast
import re

In [52]:
#path to database
#database_path = r'paris_metropolitan_evolution_db_time.geojson'
database_lines_path = r'paris_metropolitan_an_evolution_lines.geojson'
database_csv_path = 'final_database_files/paris_metropolitan_evolution_db.csv'
#load the database files
#paris_metro_db = json.load(open(database_path))
paris_metro_lines_db = json.load(open(database_lines_path))
paris_metro_xls_db = pd.read_excel('final_database_files/paris_metropolitan_evolution_db.xlsx', skiprows=1, convert_float=True)
#paris_metro_lines_xls_db = pd.read_excel('paris_metropolitan_evolution_lines_db.xlsx', skiprows=1)

## First map: 3 network layers over a contemporain basemap
The first map generated is the one displaying the data extracted from the three old maps as layers.
Stations are represented as markers on the map and popup windows with corresponding information are attached.
Lines are represented as line path on the map and also have popup windows with corresponding information.
In order to more easily generate the map, the database in xlsx format was used for the creation of the stations markers and the geojson database was used for the lines.

In [55]:
#We first create a basemap
map = folium.Map(location=[48.864716, 2.349014], tiles='cartodbpositron', zoom_start=12)

#We create a feature group for each map layer
feature_group_1908 = folium.FeatureGroup(name='1908 stations')
feature_group_1915 = folium.FeatureGroup(name='1915 stations')
feature_group_1950 = folium.FeatureGroup(name='1950 stations')

########### stations ###################################################################### 
#Itterate through the database of stations and retrieve data to be displayed
for index, station in paris_metro_xls_db.iterrows():
    #cooridnates
    lon = float(station['y'])
    lat = float(station["x"])
    
    #text to be displayed
    popup_text = "<b>Name</b>: "+ station["stop_name"].encode("ascii", "xmlcharrefreplace").decode("utf-8").replace("'", "&#39;")+"<br>"+"<b>Line</b>: "+str(station["stop_line"])
    
    #display information about station if exists
    if pd.notnull(station["stop_info"]):
        popup_text = popup_text + "<br><b>Info</b>: "+station["stop_info"].encode("ascii", "xmlcharrefreplace").decode("utf-8").replace("'", "&#39;")
    
    #if station belongs to 1908 map, add a marker to corresponding layer
    if station['start_map'] == 1908 :
        folium.Marker(
            location = [lon,lat],
            popup = popup_text+"</br><b>Opening date</b>: "+str(station['open_date'])+"</br><small><b>ratp id</b>: " + str(station['ratp_id'])+"</small>",
            icon=folium.Icon(color='green',icon='subway', prefix='fa'),
        ).add_to(feature_group_1908)
        
    #if station belongs to 1915 map, add a marker to corresponding layer
    if station['end_map'] == 1950 or station['end_map'] == 1915: 
        if station['start_map'] == 1915 or station['start_map'] == 1908 :
            folium.Marker(
                location = [lon,lat],
                popup = popup_text+"</br><b>Opening date</b>: "+str(station['open_date'])+"</br><small><b>ratp id</b>: " + str(station['ratp_id'])+"</small>",
                icon=folium.Icon(color='blue',icon='subway', prefix='fa'),
            ).add_to(feature_group_1915)
        
    #if station belongs to 1950 map, add a marker to corresponding layer
    if station['end_map'] == 1950 :
        folium.Marker(
            location = [lon,lat],
            popup = popup_text+"</br><b>Opening date</b>: "+str(station['open_date'])+"</br><small><b>ratp id</b>: " + str(station['ratp_id'])+"</small>",
            icon=folium.Icon(color='red',icon='subway', prefix='fa'),
        ).add_to(feature_group_1950)
        
########### lines ###################################################################### 
#Itterate through the JSON database of lines and retrieve data to be displayed
for feature in paris_metro_lines_db["features"] :
    #coordinates
    coord = copy.deepcopy(feature['geometry']['coordinates'])
    #text to be displayed
    popup_text = "<b>Line name</b>: "+ copy.deepcopy(feature['properties']['stop_name'])
    #if information about the line exists, then it is displayed
    if feature['properties']['info'] != None:
        popup_text = popup_text + "<br><b>Info</b>: " + copy.deepcopy(feature['properties']['stop_info'])
    #if line belongs to 1908 map, corresponding line path is added to corresponding layer
    if feature['properties']['start_map'] == 1908 :
        geojson = folium.GeoJson(feature,
                       style_function=lambda feature: {
                            'fillColor': 'green',
                            'color' : 'green',
                            'weight' : 2,
                        })
        popup = folium.Popup(popup_text)
        popup.add_to(geojson)
        geojson.add_to(feature_group_1908)
    #if line belongs to 1915 map, corresponding line path is added to corresponding layer
    if feature['properties']['end_map'] == 1950 or feature['properties']['end_map'] == 1915: 
        if feature['properties']['start_map'] == 1915 or feature['properties']['start_map'] == 1908 :
            geojson = folium.GeoJson(feature,
                       style_function=lambda feature: {
                            'fillColor': 'blue',
                            'color' : 'blue',
                            'weight' : 2,
                        })
            popup = folium.Popup(popup_text)
            popup.add_to(geojson)
            geojson.add_to(feature_group_1915)
    #if line belongs to 1950 map, corresponding line path is added to corresponding layer
    if feature['properties']['end_map'] == 1950 :
        geojson = folium.GeoJson(feature,
                       style_function=lambda feature: {
                            'fillColor': 'red',
                            'color' : 'red',
                            'weight' : 2,
                        })
        popup = folium.Popup(popup_text)
        popup.add_to(geojson)
        geojson.add_to(feature_group_1950)
        
#three feature groups are added to the map
feature_group_1908.add_to(map)
feature_group_1915.add_to(map)
feature_group_1950.add_to(map)
folium.LayerControl().add_to(map)
#map is saved to an html file
map.save('map_stations_and_lines_final.html')

map

## Second map: a growing network over time
In order to create this map, the plugin TimestampedGeoJson was used. However, it was not possible to directly use the geojson database as the information used for the plugin has to be already chronologically sorted. As we are working with data information that was added a posteriori to the database, the latter was not sorted accordingly. The xlsx database is thus used in order to more easily sort the entries according to this time information. A new dictionnary is then implemented in order to produce a pseudo JSON database that is used a parameter for the plugin. 

In [44]:
#dictionary containing relevant info of each station for the map is created
stations_dic = []

#creation of the basemap
m = folium.Map(location=[48.864716, 2.349014], tiles='cartodbpositron', zoom_start=13)

#work with xls db as the entries have to be chronologically sorted
paris_metro_xls = paris_metro_xls_db

#sort the dataframe in order to display chronologically the stations
paris_metro_xls['open_date'] = pd.to_datetime(paris_metro_xls.open_date, format='%Y%m%d')
paris_metro_xls = paris_metro_xls.sort_values(by='open_date')

#iterate throught the sorted database and implement stations_dic dictionary
for index, feature in paris_metro_xls.iterrows():
    #store coordinates
    lon = float(feature['y'])
    lat = float(feature["x"])
    #if the open_date exists, then a new entry to the dictionnary stations_dic is created
    if not pd.isnull(feature['open_date']): 
        time = feature['open_date'].strftime('%Y-%m-%d')
        #the a_station entry recreates the JSON convention
        a_station = {
        'type': 'Feature',
            'geometry': {
                'type': 'Point',
                'coordinates':[lat, lon] #coordinates of the specific station are stored
            },
            'properties': {
                'popup': feature['stop_name'], #only the name of the station is displayed in the popup
                'time': time, #the key time is added with the corresponding creation date of the station
                'icon': 'circle', #the marker shape has to be defined for each station within the dictionary
                'iconstyle': {
                    'fillColor': 'green',
                    'fillOpacity': 0.6,
                    'stroke': 'false',
                    'radius': 5
                },
                'style': {'weight': 0},
                'id': 'man'
            }
        }
        stations_dic.append(a_station)
#The plugin TimestampedGeoJson is used to generate the map
plugins.TimestampedGeoJson({
    #a pseudo JSON structure is recreated and includes the stations_dic dictionary
    'type': 'FeatureCollection',
    'features': stations_dic},
    period='P1M',
    add_last_point=True,
    auto_play=True,
    loop=False,
    max_speed=1,
    loop_button=True,
    date_options='YYYY-MM',
    time_slider_drag_update=True,
    duration='P50Y'
).add_to(m)
    
#the map is saved under html format
m.save('map_time_evolution_02.html')

m